In [2]:
import pandas as pd
import numpy as np
from keras import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from keras.wrappers.scikit_learn import KerasClassifier
from keras import metrics
from sklearn.model_selection import RandomizedSearchCV
from keras.layers import Dropout
from keras.constraints import maxnorm


df = pd.read_csv(r'C:\Users\dungeon\Desktop\kick\kicktraq.csv')

X = df.drop(columns=['status_labels','avg_backers_per_pledge_tier','funding_percentage','funding_raised'
                    ,'num_backers','num_pledge_backers','avg_pledge_amount_per_backer'])
y = df['status_labels'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20,random_state = 42,stratify=y)

batch_size = [int(x) for x in np.linspace(2,50,20)]
nb_epoch = [int(x) for x in np.linspace(start =50,stop = 500,num=50)]
optimizer = ['adam']
kernel = ['random_normal']
units = [4,8,16,32,64,128]
dropout_rate = [int(x) for x in np.linspace(0.0,1,10)]
weight = [int(x) for x in np.linspace (1,5,5)]
parameters = {'batch_size': batch_size,
             'nb_epoch': nb_epoch,
             'optimizer': optimizer,
             'kernel':kernel,
             'units': units,
              'dropout_rate':dropout_rate,
              'weight': weight
             }

# K fold validation
def build_classifier(optimizer, kernel, units,dropout_rate = 0.0,weight=0):
    # Initializing Neural Network
    classifier = Sequential()
    # First Hidden Layer
    classifier.add(Dense(units = units, activation = 'relu',kernel_constraint=maxnorm(weight),
    kernel_initializer = kernel, input_dim = 15))
    # Second Hidden Layer
    classifier.add(Dense(units = units, activation='relu',kernel_constraint=maxnorm(weight),
    kernel_initializer = kernel))
    #dropout rate
    classifier.add(Dropout(dropout_rate))
    # Output Layer
    classifier.add(Dense(1,activation='sigmoid',
    kernel_initializer= kernel))
    # Compiling NN
    classifier.compile(optimizer=optimizer,loss = 'binary_crossentropy',
    metrics=['accuracy'])
  
    return classifier


classifier = KerasClassifier(build_fn=build_classifier)

random_search = RandomizedSearchCV(estimator = classifier, param_distributions = parameters,n_iter= 100, cv =5, n_jobs = 5,random_state = 42,verbose = 2)

random_search.fit(X_train,y_train)

random_search.best_params_

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  31 tasks      | elapsed:   27.4s
C:\Users\dungeon\Anaconda3\lib\site-packages\sklearn\externals\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=5)]: Done 152 tasks      | elapsed:  2.4min
[Parallel(n_jobs=5)]: Done 355 tasks      | elapsed:  5.7min


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.


[Parallel(n_jobs=5)]: Done 500 out of 500 | elapsed:  8.5min finished


Epoch 1/1
5438/5438 [==============================] - 1s 181us/step - loss: 2.4831 - acc: 0.6054


{'weight': 3,
 'units': 64,
 'optimizer': 'adam',
 'nb_epoch': 316,
 'kernel': 'random_normal',
 'dropout_rate': 0,
 'batch_size': 34}